In [ ]:
#!/usr/local/bin/python3

from Crypto.Cipher import AES
import os

key = os.urandom(16)
with open("flag.txt", "r") as f:
    flag = f.readline().strip()
cipher = AES.new(key, AES.MODE_ECB)

flag_enc = cipher.encrypt(flag.encode())
print("Here's the encrypted flag in hex: ")
print(flag_enc.hex())
print("Alright, lemme spin up my Extremely Convenient Breaker (trademark copyright all rights reserved). ")

while True:
    ecb = input("What ciphertext do you want me to break in an extremely convenient manner? Enter as hex: ")
    try:
        ecb = bytes.fromhex(ecb)
        if not len(ecb) == 64:
            print("Sorry, it's not *that* convenient. Make your ciphertext 64 bytes please. ")
        elif ecb == flag_enc:
            print("No, I'm not decrypting the flag. ")
        else:
            print(cipher.decrypt(ecb))
    except Exception:
        print("Uh something went wrong, please try again. ")


In [4]:
import socket
from Crypto.Cipher import AES
import os
import re

# 连接到远程服务
s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('chall.lac.tf', 31180))

# 接收并解析 flag 密文
data = s.recv(4096).decode()
flag_enc_hex = data.split('\n')[1]
flag_enc = bytes.fromhex(flag_enc_hex)

# 生成一个随机密钥和 AES ECB 加密器（这里仅为模拟生成已知明文密文，实际未知）
key = os.urandom(16)
cipher = AES.new(key, AES.MODE_ECB)

# 构造一个 64 字节的已知明文
known_plaintext = b'A' * 64
# 对已知明文进行加密
known_enc = cipher.encrypt(known_plaintext)

# 将已知明文的密文和 flag 的密文按 16 字节块分割
known_blocks = [known_enc[i:i + 16] for i in range(0, len(known_enc), 16)]
flag_blocks = [flag_enc[i:i + 16] for i in range(0, len(flag_enc), 16)]

# 初始化已获取的 flag 部分
partial_flag = b""

# 逐步替换密文块进行解密
for i in range(len(flag_blocks)):
    new_ciphertext = b""
    for j in range(4):  # 64 字节共 4 个 16 字节块
        if j == i:
            new_ciphertext += flag_blocks[j]
        else:
            new_ciphertext += known_blocks[j]

    # 确保新的密文长度为 64 字节
    if len(new_ciphertext) > 64:
        new_ciphertext = new_ciphertext[:64]
    elif len(new_ciphertext) < 64:
        new_ciphertext += b'\x00' * (64 - len(new_ciphertext))

    # 将新的密文转换为十六进制字符串
    new_ciphertext_hex = new_ciphertext.hex()

    # 发送新的密文给远程服务
    s.send((new_ciphertext_hex + '\n').encode())

    # 接收远程服务的解密结果
    response = s.recv(4096).decode()

    # 使用正则表达式匹配长度为 128 的十六进制字符串
    hex_match = re.search(r'([0-9a-fA-F]{128})', response)
    if hex_match:
        hex_str = hex_match.group(1)
        if len(hex_str) == 128:
            try:
                decrypted = bytes.fromhex(hex_str)
                # 提取当前块的解密结果
                current_block_decrypted = decrypted[i * 16:(i + 1) * 16]
                partial_flag += current_block_decrypted
            except ValueError:
                print("无法将匹配到的字符串转换为十六进制，请检查。")
        else:
            print("匹配到的十六进制字符串长度不符合要求。")
    else:
        print("未在响应中找到有效的 128 字符十六进制字符串。")

# 去除可能的填充字符
try:
    full_flag = partial_flag.decode().rstrip('\x00')
    print("完整的 flag: ", full_flag)
except UnicodeDecodeError:
    print("解密结果无法正确解码，请检查。")

# 关闭连接
s.close()

未在响应中找到有效的 128 字符十六进制字符串。
未在响应中找到有效的 128 字符十六进制字符串。
未在响应中找到有效的 128 字符十六进制字符串。
未在响应中找到有效的 128 字符十六进制字符串。
完整的 flag:  
